# Import packages and define functions

In [48]:
# try to run the code on google colab...
try:
  from google.colab import drive
  drive.mount('/content/drive/')
  shared_drive_foldername = 'NUTRECON'
  root = '/content/drive/Shareddrives/{}/'.format(shared_drive_foldername)
  !pip install pandas==1.4.2
  print('Running Code in Colab')
# ...or locally
except:
  # define the local directory of the project
#   root = "C:/Users/DiogoMelo/nutreconDrive/"
  root = 'D:/FV/Projects/NUTRECON/nutreconDrive/'
  print('Running Code locally')

import sys
import pandas as pd
from glob import glob
import plotly.express as px

sys.path.append(root + 'python')
from variableCoding import Vars
import importData as load_data
import plots as myPlots
import neuroeconomics as neuroEcon


_v_ = Vars()
responses_dataPath = _v_.responses_dataPath
data_folder = responses_dataPath.split('Data')[0] + 'Data/'

cols_rename = {'Trial Type' : 'trial_type',
                'reference type' : 'ref_type', 
                'reference qt' : 'ref_qt', 
                'reference p' : 'ref_prob',
                'lottery type' : 'lott_type', 
                'lottery qt' : 'lott_qt', 
                'lottery p' : 'lott_prob',
                'choice' : 'choice',}


def get_probLottery(group):
  prob_lotteryChoice = len(group[group['choice'] == 2]) / 6
  return prob_lotteryChoice

def _subject_lotteryChoices(df, subject_id):
    
    title = 'Subject {} lottery choices for day {}'.format(subject_id, df['Day'].unique()[0])

    hover_cols = ['lottery flavor']
    hover_data = {key:True if key in hover_cols else False for key in df.columns}

    fig = px.line(df, facet_col = 'lott_type', facet_row = 'trial_type',
                x='lott_qt', y=_v_.probLotteryChoice_colName, 
                color='lott_prob',  markers=True,
                symbol = 'lott_prob', 
                color_discrete_sequence = px.colors.sequential.dense,
                hover_data = hover_data, 
                category_orders= {
                            'lottery p': sorted(list(df['lott_prob'].unique())),
                            }
                    )
    fig.update_layout(
        title = title,
        legend_title="Lottery Probability",
    )                                
    fig.update_xaxes(matches=None)

    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.layout["yaxis1"].title.text = ""

    fig.show()

def plot_subjectLotteryChoices(neuroEcon_df, subject_id):
    df = pd.DataFrame(neuroEcon_df.groupby(
                        list(cols_rename.values())[:-1] + ['Day', 'lottery flavor']
                        ).apply(
                    lambda df: get_probLottery(df)), 
                    columns = [_v_.probLotteryChoice_colName]).reset_index()
    
    df.groupby('Day').apply(lambda df: _subject_lotteryChoices(df, subject_id))

subject_code_list = list(set([s.split('\\')[-1].split('_')[0] for s in glob('{}*{}*'.format(responses_dataPath, _v_.neuroEcon_id))]))

sorted(subject_code_list)


Running Code locally


['nutre001',
 'nutre002',
 'nutre003',
 'nutre004',
 'nutre005',
 'nutre006',
 'nutre007',
 'nutre008',
 'nutre009',
 'nutre010',
 'nutre011',
 'nutre012',
 'nutre013',
 'nutre014',
 'nutre015',
 'nutre016',
 'nutre017',
 'nutre018',
 'nutre019',
 'nutre020']

# Insert Subject ID below

In [49]:
subject_id = 'nutre001'

# Check behaviour

In [50]:

neuroEcon_df = load_data.nutreconTrials(responses_dataPath, [subject_id]).rename(columns = cols_rename).replace({'money':'Money'})

mt_st_mask = neuroEcon_df['trial_type'] != 'mixed_yogurt'

neuroEcon_df = neuroEcon_df[mt_st_mask].reset_index(drop=True) # Removed mixed yogurt Trials

day1_mask = neuroEcon_df['Day'] == 1
day2_mask = neuroEcon_df['Day'] == 2
day3_mask = neuroEcon_df['Day'] == 3

plot_subjectLotteryChoices(neuroEcon_df, subject_id)

# Day 2

In [57]:
alphaMoney0 = 0
alphaCplus0 = 0.1
alphaCminus0 = -1
st_betaMoney0 = 10 # also used in model with only one beta
sFactorCplus0 = 1
sFactorCminus0= 1

x0_d2_6params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0,
      sFactorCplus0, sFactorCminus0)

df = neuroEcon_df[day2_mask].copy()

res_st, res_mt, st_iterParams_df, mt_iterParams_df = neuroEcon.stepwise_estimate(df, x0_d2_6params)
parsCI_d2 = neuroEcon.print_stepwiseModel_output(res_st, res_mt)



D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:33: RuntimeWarning:

overflow encountered in exp

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:33: RuntimeWarning:

overflow encountered in exp

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

c:\Users\Admin\anaconda3\envs\nutrecon\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning:

invalid value encountered in subtract

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:33: RuntimeWarning:

overflow encountered in exp

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

c:\Users\Admin\anaconda3\envs\nutrecon\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning:

invalid value encountered in subtract

D:\FV/Proje

Same type trials
  Optimization terminated successfully.
    - parameters: [ 0.42493863 -0.51224763 -1.02306413 -5.21141092]
  - std. error: [0.02713805 0.11991182 0.25107284 0.88294093]

Confidene intervals:
  - parameter 1: 0.425 ± 0.053
  - parameter 2: -0.512 ± 0.235
  - parameter 3: -1.023 ± 0.492
  - parameter 4: -5.211 ± 1.731

Mixed type trials
  Optimization terminated successfully.
    - parameters: [ 18.23140389 -73.90425492]
  - std. error: [ 1.2486205  36.96419193]

  Confidene intervals:
    - parameter 1: 18.231 ± 2.447
    - parameter 2: -73.904 ± 72.45


## wrap up day 2

In [ ]:
d2_df = pd.DataFrame([x0_d2_6params, parsCI_d2]).T
d2_df.columns = ['init guess', 'mean and CI']
d2_df['Day'] = 2
d2_df['comment']='negative beta and alphas do not correspond to behaviour'

# Day 3

In [60]:
alphaMoney0 = 0
alphaCplus0 = 0.2
alphaCminus0 = 0
st_betaMoney0 = 10 # also used in model with only one beta
sFactorCplus0 = 5
sFactorCminus0= -10

x0_d3_6params = (alphaMoney0, alphaCplus0, alphaCminus0, 
      st_betaMoney0,
      sFactorCplus0, sFactorCminus0)

df = neuroEcon_df[day3_mask].copy()

res_st, res_mt, st_iterParams_df, mt_iterParams_df = neuroEcon.stepwise_estimate(df, x0_d3_6params)
parsCI_d3 = neuroEcon.print_stepwiseModel_output(res_st, res_mt)



D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

c:\Users\Admin\anaconda3\envs\nutrecon\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning:

invalid value encountered in subtract

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

c:\Users\Admin\anaconda3\envs\nutrecon\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning:

invalid value encountered in subtract

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:

divide by zero encountered in log

c:\Users\Admin\anaconda3\envs\nutrecon\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning:

invalid value encountered in subtract

D:\FV/Projects/NUTRECON/nutreconDrive/python\neuroeconomics.py:262: RuntimeWarning:



Same type trials
  Optimization terminated successfully.
    - parameters: [  0.53880463  -5.42963527 -16.4468993   -4.81656848]
  - std. error: [2.68953329e-02 1.44448860e+02 4.32887744e+02 7.31446617e-01]

Confidene intervals:
  - parameter 1: 0.539 ± 0.053
  - parameter 2: -5.43 ± 283.12
  - parameter 3: -16.447 ± 848.46
  - parameter 4: -4.817 ± 1.434

Mixed type trials
  Optimization terminated successfully.
    - parameters: [  5. -10.]
  - std. error: [1. 1.]

  Confidene intervals:
    - parameter 1: 5.0 ± 1.96
    - parameter 2: -10.0 ± 1.96


## Wrap up day 3

In [ ]:
d3_df = pd.DataFrame([x0_d3_6params, parsCI_d3]).T
d3_df.columns = ['init guess', 'mean and CI']
d3_df['Day'] = 3
d3_df['comment']=''

# Day 1

In [41]:
if all(day1_mask == False):
    print('Task was not run on day 1')
else:
    alphaMoney0 = 0
    alphaCplus0 = 0
    alphaCminus0 = 0
    st_betaMoney0 = 10 # also used in model with only one beta
    sFactorCplus0 = 1
    sFactorCminus0= 1

    x0_d1_6params = (alphaMoney0, alphaCplus0, alphaCminus0, 
        st_betaMoney0,
        sFactorCplus0, sFactorCminus0)

    df = neuroEcon_df[day1_mask].copy()

    res_st, res_mt, st_iterParams_df, mt_iterParams_df = neuroEcon.stepwise_estimate(df, x0_d1_6params)
    parsCI_d1 = neuroEcon.print_stepwiseModel_output(res_st, res_mt)

    

Task was not run on day 1


## Wrap up day 1

In [62]:
if not all(day1_mask == False):
    d1_df = pd.DataFrame([x0_d1_6params, parsCI_d1]).T
    d1_df.columns = ['init guess', 'mean and CI']
    d1_df['Day'] = 1
    d1_df['comment']=''

# Save Results below

In [63]:
if all(day1_mask == False):
    sub_df = pd.concat([d2_df,d3_df])
else:
    sub_df = pd.concat([d1_df,d2_df,d3_df])

fpath = 'D:/FV/Projects/NUTRECON/Results/Subject Level/{}_parameterEstimates.csv'.format(subject_id)
sub_df.to_csv(fpath, index = False)